In [3]:
### MERGE ######  
import os
import pandas as pd
import numpy as np

# Parent folder path
parent_folder = f'/Users/kalyan/Library/CloudStorage/OneDrive-Personal/Kalyan/KK-Python/Kalyan-Jupyter-Notebooks/Capex/proc_data/'
print('Folder:' + parent_folder)

# Parent folder path
concord_folder = f'/Users/kalyan/Library/CloudStorage/OneDrive-Personal/Kalyan/KK-Python/Kalyan-Jupyter-Notebooks/Capex/raw_data/'
print('Folder:' + concord_folder)

# merge master with state code and region
df_st = pd.read_excel(os.path.join(concord_folder, 'Concordance-Tables-Capex.xlsx'), sheet_name = 'state-concord') # state numeric code and region
df2 = pd.read_excel(os.path.join(parent_folder, 'Fiscal_Merged.xlsx'))
df2 = df2.rename(columns={'Years':'Year'})
df3 = pd.read_excel(os.path.join(parent_folder, 'Gross_Merged_LF.xlsx'))
df4 = pd.read_excel(os.path.join(parent_folder, 'Merged_Net_GDP.xlsx'))

dflist =[df2,df3,df4]

for i in dflist:
    i = i.rename(columns={'Year':'FY'}, inplace = True)

cc = ['State', 'FY']

# Merge Fiscal data
merged_fiscal = pd.merge(df2, df3, on=cc, how='inner').merge(df4, on = cc, how='inner').merge(
    df_st[['st_name_rbi','st_ut_code','region']], 
         left_on = ['State'], right_on = ['st_name_rbi'], how= 'left')

# Define the common pattern to remove
pattern1 = 'Gross State Value Added by Economic Activity - '
pattern2 = 'Net State Value Added by Economic Activity - '

# Rename columns using str.replace
merged_fiscal.columns = merged_fiscal.columns.str.replace(pattern1, 'GSVA-')
merged_fiscal.columns = merged_fiscal.columns.str.replace(pattern2, 'NSVA-')

#Split values based on hyphen and save the part before hyphen in a new column
merged_fiscal['FY'] = merged_fiscal['FY'].str.split('-').str[0].str[-2:] + '-' + merged_fiscal['FY'].str.split('-').str[1].str[-2:]

print(merged_fiscal.columns)
select_cols = ['st_ut_code', 'FY', 'Gross Fiscal Deficit', 'Revenue Deficit',
       'Primary Deficit', 'Own Tax Revenue', 'Own Non-Tax Revenue',
       ' Interest Payments', 'State-wise Pension',
       'State-wise Capital Expenditure', 'Capital Outlay',
       'Social Sector Expenditure', ' Gross State Domestic Product',
       'GSVA-Agriculture', 'GSVA-Manufacturing', 'GSVA-Construction',
       'GSVA-Industry', 'GSVA-Banking and Insurance ', 'GSVA-Services ',
       'Per Capita Net State Domestic Product', 'Net State Domestic Product',
       'NSVA-Agriculture', 'NSVA-Manufacturing', 'NSVA-Construction',
       'NSVA-Industry', 'NSVA-Banking and Insurance', 'NSVA-Services ',]
    
print(merged_fiscal['State'].unique())

df1 = pd.read_excel(os.path.join(parent_folder, 'Capex-Projects-Total.xlsx'))

merged_capex_fiscal = pd.merge(df1, merged_fiscal[select_cols], on=['st_ut_code','FY'], how='left')
print(merged_capex_fiscal.shape)

# Convert fiscal years to corresponding years
merged_capex_fiscal['Year'] = [2000 + int(year.split('-')[0]) for year in merged_capex_fiscal['FY']]

### Write Merged file to Excel 
out_folder_path = '/Users/kalyan/Library/CloudStorage/OneDrive-Personal/Kalyan/KK-Python/Kalyan-Jupyter-Notebooks/Capex/proc_data/'
out_file_name = 'Final_Capex.xlsx'
out_file_path = os.path.join(out_folder_path, out_file_name)

merged_capex_fiscal.to_excel((out_file_path), index=False)
print(merged_capex_fiscal.columns)

print('FINITO')


Folder:/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/proc_data/
Folder:/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/raw_data/
Index(['FY', 'State', 'Gross Fiscal Deficit', 'Revenue Deficit',
       'Primary Deficit', 'Own Tax Revenue', 'Own Non-Tax Revenue',
       ' Interest Payments', 'State-wise Pension',
       'State-wise Capital Expenditure', 'Capital Outlay',
       'Social Sector Expenditure', ' Gross State Domestic Product',
       'GSVA-Agriculture', 'GSVA-Manufacturing', 'GSVA-Construction',
       'GSVA-Industry', 'GSVA-Banking and Insurance ', 'GSVA-Services ',
       'Per Capita Net State Domestic Product', 'Net State Domestic Product',
       'NSVA-Agriculture', 'NSVA-Manufacturing', 'NSVA-Construction',
       'NSVA-Industry', 'NSVA-Banking and Insurance', 'NSVA-Services ',
       'st_name_rbi', 'st_ut_code', 'region'],
      dtype='object')
['Andhra Prades